In [2]:
import json
import requests
import pandas as pd
from tqdm import tqdm_notebook
from json import JSONDecodeError
from pyquery import PyQuery as pq

In [3]:
url = "https://eda.ru/recepty?page={}"
res = requests.get(url.format(1))

In [4]:
recipes_subcat_list = []
for recipes_cat in pq(res.text).find(".seo-footer .seo-footer__list"):
    recipes_subcats = pq(recipes_cat).find("li.seo-footer__list-item, li.seo-footer__list-title _empty")
    for recipes_subcat in recipes_subcats:
        d = {
            "title": pq(recipes_subcat).text().split("\xa0\xa0")[0],
            "href": pq(recipes_subcat).find("a").attr("href"),
            "num": int(pq(recipes_subcat).text().split("\xa0\xa0")[1])
        }
        recipes_subcat_list.append(d)

In [9]:
len(recipes_subcat_list)

258

In [7]:
data = {}

for rec_cat in recipes_subcat_list:
    print(rec_cat["title"])
    has_items = True
    page = 1
    
    while has_items:
        res = requests.get("https://eda.ru/{cat}?page={page}".format(cat=rec_cat["href"], page=page)).text
        
        if len(pq(res).find(".recipes-page__recipes .tile-list__horizontal-tile")):
            page += 1
            for recipe in pq(res).find(".recipes-page__recipes .tile-list__horizontal-tile"):
                title = pq(recipe).find(".horizontal-tile__item-title").text()
                href = pq(recipe).find(".horizontal-tile__item-title a").attr("href")
                ingredients = []
                for ingredient in pq(recipe).find(".ingredients-list .ingredients-list__content-item"):
                    try:
                        attr = pq(ingredient).attr("data-ingredient-object")
                        ingredients.append(json.loads(attr.replace('""', '"')))
                    except JSONDecodeError as err:
                        print(pq(ingredient).attr("data-ingredient-object"))
                        break
                
                data[href] = {
                    "title": title,
                    "ingredients": ingredients,
                    "category": rec_cat["title"]
                }
        else:
            has_items = False

Пирожное
Маффины
Пудинг
Пирожки
Чизкейк
Булочки
Конфеты
Мусс
Суфле
Сладкие рулеты
Брауни
Шарлотка
Бисквит
Лепешки
Киш
Крем
Тарталетки
Капкейк
Тирамису
Куличи
Панкейки
Штрудель
Вафли
Сорбет
Тесто для пирожков
Пончики
Песочное печенье
Пряники
Хачапури
Гренки
Манник
Галеты
Бискотти
Эклеры
Крекеры
Круассаны
Слойки
Ватрушки
Зефир
Рогалики
Творожный торт
Пасха
Фокачча
Коврижка
Курник
Трубочки
Пахлава
Профитроли
Творожники
Глазурь
Коржики
Пастила
Хворост
Бублики
Пирожное «Мадлен»
Плюшки
Самса
Беляши
Песочный торт
Ачма
Торт-суфле
Кулебяка
Кнедлики
Расстегаи
Запеканка
Котлеты
Пюре
Лапша
Шашлык
Стейки
Плов
Роллы
Жаркое
Лазанья
Перец фаршированный
Вареники
Овощное рагу
Тефтели
Фрикадельки
Гратен
Голубцы
Отбивные
Пельмени
Деруны
Гуляш
Жульен
Паэлья
Соте
Кебаб
Рататуй
Зразы
Шницель
Шаурма
Мусака
Буррито
Клецки
Биточки
Манты
Бифштекс
Чахохбили
Бефстроганов
Цыпленок табака
Долма
Фрикасе
Хинкали
Эскалоп
Люля-кебаб
Мясо по-французски
Буженина
Аджапсандал
Азу
Галушки
Печеночный торт
Бешбармак
Чанахи
Бри

In [10]:
len(data)

13513

In [11]:
for num, d in enumerate(data):
    data[d]["id"] = num

In [12]:
json.dump(data, open("recipes.json", "wt", encoding="utf8"))

In [12]:
data_new = json.load(open("recipes.json", "rt", encoding="utf8"))

In [13]:
df = pd.DataFrame.from_dict(data_new, orient="index")

In [15]:
df

,title,img_src,ingredients,booked,likes,dislikes,time_to_cook,portions,category,id
/recepty/zakuski/ogurci-v-ostrom-marinade-16230,Огурцы в остром маринаде,https://eda.ru/img/eda/c285x285i/s2.eda.ru/Sta...,"[{'id': 13955, 'name': 'Рисовый уксус', 'amoun...",2552,120,19,5 минут,6 порций,Соленья и консервация,0
/recepty/zakuski/solenie-limoni-29601,Соленые лимоны,https://eda.ru/img/eda/c285x285i/s1.eda.ru/Sta...,"[{'id': 13427, 'name': 'Лимон', 'amount': '1 к...",929,38,8,10 минут,12 порций,Соленья и консервация,1
/recepty/sousy-marinady/marinovannyy-chesnok-so-svekloy-137101,Маринованный чеснок со свеклой,https://eda.ru/img/eda/c285x285i/s1.eda.ru/Sta...,"[{'id': 13430, 'name': 'Чеснок', 'amount': '4 ...",109,5,0,20 минут,10 порций,Соленья и консервация,2
/recepty/zagotovki/marinovannye-opyata-94048,Маринованные опята,https://eda.ru/img/eda/c285x285i/s2.eda.ru/Sta...,"[{'id': 14584, 'name': 'Опята', 'amount': '1 к...",88,1,1,40 минут,10 порций,Соленья и консервация,3
/recepty/zakuski/marinovannyy-arbuz-38167,Маринованный арбуз,https://eda.ru/img/eda/c285x285i/s1.eda.ru/Sta...,"[{'id': 13421, 'name': 'Соль', 'amount': '100 ...",39,1,0,15 минут,6 порций,Соленья и консервация,4
...,...,...,...,...,...,...,...,...,...,...
/recepty/vypechka-deserty/dieticheskoe-pirozhnoe-kartoshka-93042,Диетическое пирожное «Картошка»,https://eda.ru/img/eda/c285x285i/s1.eda.ru/Sta...,"[{'id': 14087, 'name': 'Нут', 'amount': '130 г...",23,1,0,1 час 30 минут,10 порций,Пирожное,17892
/recepty/vypechka-deserty/pirozhnoe-kartoshka-iz-temnogo-shokolada-s-dobavleniem-konjaka-46098,Пирожное «Картошка» из темного шоколада с…,https://eda.ru/img/eda/c285x285i/s1.eda.ru/Sta...,"[{'id': 16487, 'name': 'Печенье «Топленое моло...",53,2,0,2 часа,10 порций,Пирожное,17893
/recepty/vypechka-deserty/shokoladno-limonnie-pirozhnie-46552,Шоколадно-лимонные пирожные,https://eda.ru/img/eda/c285x285i/s2.eda.ru/Sta...,"[{'id': 13458, 'name': 'Пшеничная мука', 'amou...",26,1,0,40 минут,6 порций,Пирожное,17894
/recepty/vypechka-deserty/krubnichno-jogurtovie-pirozhnie-36119,Крубнично-йогуртовые пирожные,https://eda.ru/img/eda/c285x285i/s2.eda.ru/Sta...,"[{'id': 13458, 'name': 'Пшеничная мука', 'amou...",32,1,1,1 час,6 порций,Пирожное,17895
